# Linear Elasticity

In [1]:
using Gridap
using Plots
using GridapMakie
using CairoMakie 
using GLMakie
using Gridap.Geometry
using GridapGmsh
using Gmsh: Gmsh, gmsh

In this tutorial we will go through the modeling of a linear elasticity problem on a 3D domain. We will do this by considering a structural element called a **link plate**. A link plate is often used to connect two structural elements using bolts, where through each hole there goes a bolt. 

<div style="text-align: center;">
  <img src="./images/link_plate.png" alt="Link plate" width="450"/>
</div>

We want to find the **displacement vector** $\mathbf{u}(\mathbf{x}) = [u_{1}(\mathbf{x}), u_{2}(\mathbf{x}), u_{3}(\mathbf{x})]$ where the subnotations $1, 2 \: \& \: 3$ are for the 3 Cartesian directions. Therefore, we can already see that our unknown quantity is a vector-valued unknown, which we previously discussed in the first tutorial as the second kind of quantities we may deal with (scalar-valued quantites were the first type we discussed and often the easiest to deal with). 

Let us recall our approach for solving problems using FEM. We first need to formulate the problem by describing the PDEs. Simultaneously, we define the regions of the domain where we apply the boundary conditions. 

<div style="text-align: center;">
  <img src="./images/link_plate_BCs.png" alt="Link plate" width="450"/>
</div>

We impose the following boundary conditions:

- All components of the displacement vector are constrained to **zero** on the surface $\Gamma_G$ (marked in green in the figure).
- On the surface $\Gamma_B$ (marked in blue), the **first component** of the displacement vector is prescribed to the value $\delta = 5\,\text{mm}$.
- No body or surface forces are included in this example.

Formally, the Partial Differential Equation (PDE) to solve is:

$$
\begin{aligned}
-\nabla \cdot \sigma(\mathbf{u}) &= 0 && \text{in } \Omega, \\
\mathbf{u} &= 0 && \text{on } \Gamma_G, \\
u_1 &= \delta && \text{on } \Gamma_B, \\
\sigma(\mathbf{u}) \cdot \mathbf{n} &= 0 && \text{on } \Gamma_N.
\end{aligned}
$$

## Definitions

- **Displacement vector ($\mathbf{u}$)**: The unknown displacement vector.

- **Unit outward normal ($\mathbf{n}$)**: The unit outward normal to the Neumann boundary $\Gamma_N$, where:

$$
\Gamma_N = \partial \Omega \setminus (\Gamma_B \cup \Gamma_G).
$$

- **Stress tensor ($\sigma(\mathbf{u})$)**: Defined as:

$$
\sigma(\mathbf{u}) = \lambda \, \text{tr}(\varepsilon(\mathbf{u})) \mathbf{I} + 2 \mu \varepsilon(\mathbf{u}),
$$

where:
  - $\mathbf{I}$: The identity tensor (2nd order),
  - $\lambda$ and $\mu$: The Lamé parameters,
  - **Strain tensor ($\varepsilon(\mathbf{u})$)**, defined as:

$$
\varepsilon(\mathbf{u}) = \frac{1}{2} \left( \nabla \mathbf{u} + (\nabla \mathbf{u})^T \right).
$$

### Material Parameters

For this example, we consider aluminum with the following material properties:

- **Young's modulus**: $E = 70 \cdot 10^9 \, \text{Pa}$,
- **Poisson's ratio**: $\nu = 0.33$.

From these values, the Lamé parameters are calculated as:

$$
\lambda = \frac{E \nu}{(1 + \nu)(1 - 2 \nu)}, \quad
\mu = \frac{E}{2(1 + \nu)}.
$$


In [2]:
model = GmshDiscreteModel("msh_files/solid.msh")

Info    : Reading 'msh_files/solid.msh'...
Info    : 188 entities
Info    : 10257 nodes
Info    : 41802 elements
Info    : Done reading 'msh_files/solid.msh'


UnstructuredDiscreteModel()

In [8]:
const E = 70.0e9
const ν = 0.33
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))
σ(ε) = λ*tr(ε)*one(ε) + 2*μ*ε

σ (generic function with 1 method)

In [3]:
# Get the labels of the entities
labels = get_face_labeling(model)
initial_tags = get_tag_entities(labels)[end]
tag_from_names = get_tag_from_name(labels)

Dict{String, Int64} with 6 entries:
  "surface_1"   => 4
  "surface_1_c" => 1
  "surface_2_c" => 3
  "material_1"  => 5
  "surface_2"   => 2
  "material_2"  => 6

In [4]:
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3, Float64}, order)

V0 = TestFESpace(model, reffe; conformity=:H1, dirichlet_tags=["surface_1","surface_2"])

UnconstrainedFESpace()

In [5]:
g1(x) = VectorValue(0.005,0.0,0.0)
g2(x) = VectorValue(0.0,0.0,0.0)

g2 (generic function with 1 method)

In [7]:
U = TrialFESpace(V0, [g1,g2])

TrialFESpace()

In [9]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [11]:
a(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ
l(v) = 0

op = AffineFEOperator(a,l,U,V0)
uh = solve(op)

SingleFieldFEFunction():
 num_cells: 40144
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10569126572797562389

In [12]:
writevtk(Ω,"linear_elasticity",cellfields=["uh"=>uh,"epsi"=>ε(uh),"sigma"=>σ∘ε(uh)])

(["linear_elasticity.vtu"],)

<div style="text-align: center;">
  <img src="./images/link_plate_solution.png" alt="Link plate" width="450"/>
</div>